# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [2]:
import sqlite3

In [18]:
con = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.executemany('''
SELECT FirstName,Lastname 
FROM Customer 
Where country = ? 
''', [('Canada',),('USA',)])
#1. проитерировать курсор
#2. .fetchone()
#3. .fetchmany(m)
#4. .fetchall()
print(cur.fetchall())
cur.close()
con.close()

ProgrammingError: executemany() can only execute DML statements.

2. Найти и вывести на экран названия всех альбомов группы Accept

In [22]:
con = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.execute('''
SELECT Artist.Name,Album.Title
FROM Artist,Album
WHERE  Artist.Artistid = Album.Artistid AND Artist.Name = 'Accept'
''')

print(cur.fetchall())
cur.close()
con.close()

[('Accept', 'Balls to the Wall'), ('Accept', 'Restless and Wild')]


3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [17]:
con = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = con.cursor()
cur.execute('''
CREATE TABLE  Student(
    id INTEGER,
    name VARCHAR(50)
)
''')
cur.execute('''
INSERT INTO Student(id, name)
VALUES (1,'Nikita'),(2,'Vlad')
''')
con.commit()
cur.close()
con.close()

[]


## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [1]:
import sqlite3

con = sqlite3.connect('data/recipes.db')
cur = con.cursor()

cur.close()
con.close()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [6]:
con = sqlite3.connect('data/recipes.db')
cur = con.cursor()
cur.execute('''
CREATE TABLE  Recipe(
    id INTEGER,
    name TEXT,
    minutes INTEGER,
    submitted DATE,
    description TEXT,
    n_ingredients INTEGER)
''')
con.commit()
cur.close()
con.close()

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [7]:
con = sqlite3.connect('data/recipes.db')
cur = con.cursor()
cur.execute('''
CREATE TABLE  Review(
    id INTEGER,
    user_id INTEGER,
    recipe_id INTEGER,
    date DATE,
    rating REAL,
    review TEXT,
    PRIMARY KEY (id))
''')

cur.execute('''
SELECT * FROM Review JOIN Recipe ON Recipe.id = Review.recipe_id
''')
print(cur.fetchall())
con.commit()
cur.close()
con.close()

[]


4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [9]:
from sqlalchemy import create_engine
import pandas as pd
recipes = pd.read_csv('data/recipes_sample_with_tags_ingredients.csv',sep = ',',parse_dates=['submitted'],index_col = 'id')[['name', 'minutes', 'submitted', 'description', 'n_ingredients']]
reviews = pd.read_csv('data/reviews_sample.csv',sep = ',',parse_dates=['date']).rename(columns={'Unnamed: 0':'id'})[['id','user_id','recipe_id', 'date', 'rating', 'review']]
engine = create_engine('sqlite:///data/recipes.db', echo=False)
with engine.begin() as connection:
    recipes.to_sql('recipe', con=connection, if_exists='append')
with engine.begin() as connection:
    reviews.to_sql('review', con=connection, if_exists='append',index=False)

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [13]:
con = sqlite3.connect('data/recipes.db')
cur = con.cursor()

cur.execute('''
SELECT id,name,n_ingredients FROM Recipe 
WHERE n_ingredients=10
''')
print(cur.fetchmany(10))
cur.close()
con.close()

[(125195, 'kelly s creamy cheddar pea salad', 10), (120297, 'middle eastern   twice baked potatoes', 10), (342620, 'secret ingredient  bbq meatloaf', 10), (325714, 'the  carrot cake', 10), (437637, 'just peachy  cobbler', 10), (376813, 'magic  chicken noodle soup', 10), (401605, 'missy s  sweet potato casserole', 10), (38852, 'second only to my meatloaf  meatballs', 10), (198343, 'we re going bananas  bread', 10), (129919, 'wolf creek inn  macaroni   cheese', 10)]


6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [14]:
con = sqlite3.connect('data/recipes.db')
cur = con.cursor()

cur.execute('''
SELECT name,MAX(minutes) FROM Recipe 
''')
print(cur.fetchone())
cur.close()
con.close()

('blueberry liqueur', 129615)


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [26]:
id = int(input('Введите id: '))
con = sqlite3.connect('data/recipes.db')
cur = con.cursor()

cur.execute(f'''
SELECT * FROM Recipe
WHERE id = {id}
''')
if(str(cur.fetchone())=='None'):
    print('Рецепта с таким id нет')
else:
    print(cur.fetchone())
cur.close()
con.close()

Введите id: 125195
(125195, 'kelly s creamy cheddar pea salad', 20, '2005-06-09 00:00:00.000000', "i'm not a big fan of peas, but like them in things. this salad is one of my favorites. happy to share!", 10)


8. Найдите кол-во отзывов с рейтингом 5.

In [65]:
con = sqlite3.connect('data/recipes.db')
cur = con.cursor()

cur.execute('''
SELECT count(id) FROM review
WHERE rating = 5
''')
print(cur.fetchone())
cur.close()
con.close()

(91361,)


9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [7]:
con = sqlite3.connect('data/recipes.db')
cur = con.cursor()

cur.execute('''
SELECT count(my_count)
FROM (SELECT min(rating) as my_count FROM review
GROUP BY recipe_id)
WHERE my_count>4
''')
print(cur.fetchone())
cur.close()
con.close()

(12715,)


10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [58]:
con = sqlite3.connect('data/recipes.db')
cur = con.cursor()

cur.execute('''
SELECT count(id) FROM recipe
WHERE strftime('%Y',submitted)= '2010' AND minutes>=15
''')
print(cur.fetchone())
cur.close()
con.close()

(2638,)


11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [62]:
con = sqlite3.connect('data/recipes.db')
cur = con.cursor()

cur.execute('''
SELECT recipe_id,Recipe.name,user_id,submitted,rating FROM Review,Recipe
WHERE Review.recipe_id=Recipe.id AND recipe.n_ingredients>=3
ORDER BY recipe_id
''')
print(cur.fetchmany(10))
cur.close()
con.close()

[(48, 'boston cream pie', 32421, '1999-08-24 00:00:00.000000', 0.0), (48, 'boston cream pie', 68674, '1999-08-24 00:00:00.000000', 2.0), (48, 'boston cream pie', 32421, '1999-08-24 00:00:00.000000', 0.0), (48, 'boston cream pie', 68674, '1999-08-24 00:00:00.000000', 2.0), (55, 'betty crocker s southwestern guacamole dip', 53959, '1999-09-15 00:00:00.000000', 4.0), (55, 'betty crocker s southwestern guacamole dip', 165567, '1999-09-15 00:00:00.000000', 5.0), (55, 'betty crocker s southwestern guacamole dip', 851190, '1999-09-15 00:00:00.000000', 5.0), (55, 'betty crocker s southwestern guacamole dip', 1060485, '1999-09-15 00:00:00.000000', 5.0), (55, 'betty crocker s southwestern guacamole dip', 53959, '1999-09-15 00:00:00.000000', 4.0), (55, 'betty crocker s southwestern guacamole dip', 165567, '1999-09-15 00:00:00.000000', 5.0)]
